<a href="https://www.kaggle.com/code/avinashrola/titanic-6-algo?scriptVersionId=107809812" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic1/train.csv
/kaggle/input/titanic1/test.csv
/kaggle/input/titanic1/gender_submission.csv
/kaggle/input/test-file/tested.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [3]:
data_train = pd.read_csv('/kaggle/input/titanic1/train.csv')
data_train.head()
data_train.shape

(891, 12)

In [4]:
data_test = pd.read_csv('/kaggle/input/titanic1/test.csv')
data_test.head()
data_test.shape

(418, 11)

In [5]:
# check the duplicate values
data_train.duplicated().sum()

0

In [6]:
# Check the null values
data_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
# We will replace null values of Cabin using mode of cabin values

data_train['Cabin'] = data_train['Cabin'].fillna(data_train['Cabin'].mode())

In [8]:
data_train['Embarked'].mode()

0    S
dtype: object

In [9]:
# We will replace the null values in Embarked with mode of Embarked 

data_train['Embarked'] = data_train['Embarked'].fillna(data_train['Embarked'].mode()) 

In [10]:
# We will replace the null values in Age with mean of Age 

data_train['Age'] = data_train['Age'].fillna(data_train['Age'].mean()) 

In [11]:
data_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          685
Embarked         2
dtype: int64

In [12]:
# Removes all unnecessary columns

data_train = data_train[['Survived','Pclass','Sex','Age','SibSp','Parch']]
data_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [13]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 41.9+ KB


In [14]:
data_train['Age'].mean()

29.69911764705882

In [15]:
# convert Survived column category from float to int

data_train['Survived'] = data_train['Survived'].astype('int64')

In [16]:
# Convert data type of Age from string to numerical

data_train['Age'] = data_train['Age'].astype('int64')



In [17]:
# Convert the Categorical column 'Sex' to Numerical and store it to new column

data_train['Is male'] = pd.get_dummies(data_train['Sex'], drop_first=True)
data_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Is male
0,0,3,male,22,1,0,1
1,1,1,female,38,1,0,0
2,1,3,female,26,0,0,0
3,1,1,female,35,1,0,0
4,0,3,male,35,0,0,1


In [18]:
# Drop the Sex Column

data_train.drop('Sex', axis=1, inplace=True)

In [19]:
data_train.head()

,Survived,Pclass,Age,SibSp,Parch,Is male
0,0,3,22,1,0,1
1,1,1,38,1,0,0
2,1,3,26,0,0,0
3,1,1,35,1,0,0
4,0,3,35,0,0,1


In [20]:
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [21]:
# Removes all unnecessary columns

data_test = data_test[['Pclass','Sex','Age','SibSp','Parch']]
data_test.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,male,34.5,0,0
1,3,female,47.0,1,0
2,2,male,62.0,0,0
3,3,male,27.0,0,0
4,3,female,22.0,1,1


In [22]:
# Convert the Categorical column 'Sex' to Numerical and store it to new column

data_test['Is male'] = pd.get_dummies(data_test['Sex'], drop_first=True)
data_test.head()

,Pclass,Sex,Age,SibSp,Parch,Is male
0,3,male,34.5,0,0,1
1,3,female,47.0,1,0,0
2,2,male,62.0,0,0,1
3,3,male,27.0,0,0,1
4,3,female,22.0,1,1,0


In [23]:
# Drop the Sex Column

data_test.drop('Sex', axis=1, inplace=True)

In [24]:
data_test.isnull().sum()

Pclass      0
Age        86
SibSp       0
Parch       0
Is male     0
dtype: int64

In [25]:
# We will replace the null values in Age with mean of Age 

data_test['Age'] = data_test['Age'].fillna(data_test['Age'].mean()) 

In [26]:
# Convert data type of Age from float to numerical

data_test['Age'] = data_test['Age'].astype('int64')

In [27]:
data_test.head()

,Pclass,Age,SibSp,Parch,Is male
0,3,34,0,0,1
1,3,47,1,0,0
2,2,62,0,0,1
3,3,27,0,0,1
4,3,22,1,1,0


In [28]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Pclass   418 non-null    int64
 1   Age      418 non-null    int64
 2   SibSp    418 non-null    int64
 3   Parch    418 non-null    int64
 4   Is male  418 non-null    uint8
dtypes: int64(4), uint8(1)
memory usage: 13.6 KB


# **Split the data**

In [29]:
x = data_train.drop('Survived', axis=1)
y = data_train['Survived']
print(x.head())
print('=======================')
print(y.head())

   Pclass  Age  SibSp  Parch  Is male
0       3   22      1      0        1
1       1   38      1      0        0
2       3   26      0      0        0
3       1   35      1      0        0
4       3   35      0      0        1
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [30]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

# **Model Building**

**1. Logistic Regression**

In [31]:
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [32]:
y_pred1 = lr.predict(data_test)

In [33]:
# training data score
print (lr.score(x_train,y_train))

0.773876404494382


In [34]:
# test data score
print (lr.score(x_test,y_test))

0.8547486033519553


**2. Naive Bayes**

In [35]:
nb = GaussianNB()
nb.fit(x_train,y_train)
y_pred2 = nb.predict(x_test)

In [36]:
# training data score
print (nb.score(x_train,y_train))

0.7752808988764045


In [37]:
# test data score
print (nb.score(x_test,y_test))

0.8491620111731844


In [38]:
accuracy_score(y_pred2,y_test)

0.8491620111731844

**3. KNN**

In [39]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
y_pred3 = knn.predict(x_test)

In [40]:
# training data score
print (knn.score(x_train,y_train))

0.848314606741573


In [41]:
# test data score
print (knn.score(x_test,y_test))

0.7597765363128491


In [42]:
accuracy_score(y_pred3,y_test)

0.7597765363128491

**4. Decision Tree**

In [43]:
dt = DecisionTreeClassifier(criterion='entropy',max_depth=3)
dt.fit(x_train,y_train)
y_pred4 = dt.predict(x_test)

In [44]:
# training data score
print (dt.score(x_train,y_train))

0.8117977528089888


In [45]:
# test data score
print (dt.score(x_test,y_test))

0.8324022346368715


In [46]:
accuracy_score(y_pred4,y_test)

0.8324022346368715

**5. Random Forest**

In [47]:
rf = RandomForestClassifier(n_estimators=200,min_samples_split=8,min_samples_leaf=5,n_jobs=5)
rf.fit(x_train,y_train)
y_pred5 = rf.predict(x_test)

In [48]:
# training data score
print (rf.score(x_train,y_train))

0.8525280898876404


In [49]:
# test data score
print (rf.score(x_test,y_test))

0.8324022346368715


In [50]:
accuracy_score(y_pred5,y_test)

0.8324022346368715

**6.SVM**

In [51]:
svm_model = svm.SVC(kernel='linear')
svm_model.fit(x_train,y_train)
y_pred6 = svm_model.predict(x_test)

In [52]:
# training data score
print (svm_model.score(x_train,y_train))

0.776685393258427


In [53]:
# test data score
print (svm_model.score(x_test,y_test))

0.8268156424581006


In [54]:
accuracy_score(y_pred6,y_test)

0.8268156424581006

In [55]:
print(f'Logistic Regression : {lr.score(x_test,y_test)*100}')
print(f'Naive Bayes : {accuracy_score(y_pred2,y_test)*100}')
print(f'KNN : {accuracy_score(y_pred3,y_test)*100}')
print(f'Decision Tree : {accuracy_score(y_pred4,y_test)*100}')
print(f'Random Forest : {accuracy_score(y_pred5,y_test)*100}')
print(f'SVM : {accuracy_score(y_pred6,y_test)*100}')

Logistic Regression : 85.47486033519553
Naive Bayes : 84.91620111731844
KNN : 75.97765363128491
Decision Tree : 83.24022346368714
Random Forest : 83.24022346368714
SVM : 82.68156424581005
